#Data Preperation

https://drive.google.com/drive/folders/1A8DZKkRCjS7oX5Y0ZnPSp114jYvqCOUZ?usp=sharing

## Cleaning 

In [ ]:
import os
import sys
import subprocess
import numpy as np
import glob
import json
import pandas as pd

'''
This script creates a folder "Extracted_data" inside which it extracts all the wav files in the directories date-wise
'''

coswara_data_dir = os.path.abspath('../input/coswaracovidcoughdata') # Local Path of iiscleap/Coswara-Data Repo
extracted_data_dir = os.path.abspath('Extracted_data')  

if not os.path.exists(coswara_data_dir):
    raise("Check the Coswara dataset directory!")

if not os.path.exists(extracted_data_dir):
    os.makedirs(extracted_data_dir) # Creates the Extracted_data folder if it doesn't exist

dirs_extracted = set(map(os.path.basename,glob.glob('{}/202*'.format(extracted_data_dir))))
dirs_all = set(map(os.path.basename,glob.glob('{}/202*'.format(coswara_data_dir))))

dirs_to_extract = list(set(dirs_all) - dirs_extracted)

for d in dirs_to_extract:
    p = subprocess.Popen('cat {}/{}/*.tar.gz.* |tar -xvz -C {}/'.format(coswara_data_dir, d, extracted_data_dir), shell=True)
    p.wait()


print("Extraction process complete!")

In [ ]:
!ls ./Extracted_data/*/*.csv

In [ ]:
file_paths = []
for files in glob.glob("./Extracted_data/*/*/*.csv"):
    print(files)
    file_paths.append(files) 

In [ ]:
wav_paths = []
for f in glob.glob("./Extracted_data/*/*/breathing-deep.wav"):
#   print(f)
  wav_paths.append(f)

In [ ]:
len(wav_paths), len(file_paths)

In [ ]:
pd.read_csv(file_paths[0]).head()

In [ ]:
combined_csv = pd.read_csv('../input/coswaracovidcoughdata/combined_data.csv')

In [ ]:
# combined_csv = pd.concat([pd.read_csv(f) for f in file_paths])
combined_csv.head()

In [ ]:
test_result_csv = combined_csv.loc[combined_csv.test_status!='na']
test_result_csv

In [ ]:
cleaned_data = test_result_csv.drop(['test_date', 'l_l', 'l_c', 'l_s', 'ep'], axis=1)
cleaned_data

In [ ]:
paths_lis = cleaned_data.id.unique()  
paths_lis[0]

In [ ]:
wav_paths1 = []
# wav_paths2 = []
for i in range(len(paths_lis)):
  for f in glob.glob("./Extracted_data/*/"+paths_lis[i]+"/cough-heavy.wav"):
    print(f)
    wav_paths1.append(f)
#   for fi in glob.glob("./Extracted_data/*/"+paths_lis[i]+"/cough-shallow.wav"):
#     print(fi)
#     wav_paths2.append(fi)

In [ ]:
len(wav_paths1)
# len(wav_paths2)

In [ ]:
cleaned_data.reset_index(drop=True, inplace=True)

In [ ]:
# wav1 = deepcough, wav2 = shallowcough
cleaned_data['wav1_path'] = pd.DataFrame(wav_paths1, index=None)
# cleaned_data['wav2_path'] = pd.DataFrame(wav_paths2, index=None)
cleaned_data

In [ ]:
final_csv = cleaned_data.reindex(columns=['id', 'wav1_path', 'a', 'covid_status', 'dT', 'fV', 'g', 'rU', 'test_status', 'um',
       'cough', 'cold', 'fever', 'loss_of_smell', 'diabetes', 'st', 'bd', 'ht',
       'pneumonia', 'mp', 'ftg', 'smoker', 'ctScan', 'vacc', 'testType', 'ihd',
       'asthma', 'others_preexist', 'ctDate', 'ctScore', 'diarrhoea', 'cld',
       'others_resp'])
final_csv.head()

In [ ]:
final_csv.iloc[:1299]

In [ ]:
# final_csv.to_csv('/content/drive/MyDrive/Colab Notebooks/Coswara/cleaned_data.csv')
# final_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Coswara/cleaned_data.csv')

## wav to specgram

In [ ]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import soundfile as sf
import pandas as pd
from tqdm import tqdm_notebook as tq
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# X, sample_rate = librosa.load(wav_paths1[10])
from scipy.io import wavfile

try:
    count = 0
    for x in final_csv.wav1_path:
        y, sample_rate = librosa.load(x)
    if samplerate is not None:
        count += 1
    #         print(samplerate)
except ValueError:
    pass
        
print(count)

In [ ]:
def feature_extraction(file_name):
    try:
        X, sr = librosa.load(file_name)
        if X.ndim > 1:
            X = X[:, 0]
        X = X.T
        stft = np.abs(librosa.stft(X))
        chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        rmse = np.mean(librosa.feature.rms(y=X))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=X, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(X))
        mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T, axis=0)  # 40 values
#         mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
        return chroma_stft, rmse, spec_cent, spec_bw, rolloff, zcr, mfcc 
    except ValueError:
        return None

    # Get features
#     stft = np.abs(librosa.stft(X))
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)  # 40 values
    # zcr = np.mean(librosa.feature.zero_crossing_rate)
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, # tonal centroid features
#                       axis=0)

    # Return computed features
#     return mfcc, mel
#     return mfccs, chroma, mel, contrast, tonnetz

In [ ]:
features = pd.DataFrame([feature_extraction(j) for j in final_csv.wav1_path], columns = ['chroma_stft', 'rmse', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc'])
# features1 = pd.DataFrame([feature_extraction(j) for j in final_csv.wav1_path], columns = ['mfccs1', 'chroma1', 'mel1', 'contrast1', 'tonnetz1'], dtype='object')
# features2 = pd.DataFrame([feature_extraction(k) for k in final_csv.wav2_path], columns = ['mfccs2', 'chroma2', 'mel2', 'contrast2', 'tonnetz2'], dtype='object')

In [ ]:
features.count(), final_csv.count()

In [ ]:
final_csv[['id', 'a', 'covid_status', 'g', 'test_status']]

In [ ]:
features

In [ ]:
featured_csv = pd.concat([final_csv[['id', 'a', 'covid_status', 'g', 'test_status']], features], axis=1)
featured_csv

In [ ]:
type(featured_csv.mfcc[0])

In [ ]:
featured_csv.to_pickle('./features_tab.pkl')

In [ ]:
featured_csv.zcr

In [ ]:
specgram(fet[3], Fs = 1, cmap = plt.cm.bone)[~1][:]

## image folders 


In [ ]:
import pandas as pd

df = pd.read_pickle('./features_tab.pkl')
df

In [ ]:
sample_rate = 22050
rate = 48000

In [ ]:
fig,ax = plt.subplots(1)
fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
ax.axis('off')
ax.specgram(df.mel[5], Fs = rate, cmap = plt.cm.bone)[0][0]
# fig.savefig('sp_xy.png', bbox_inches='tight', pad_inches = 0)

In [ ]:
df.groupby('covid_status').count()

In [ ]:
df.groupby('test_status').count()

In [ ]:
!mkdir posCls
!mkdir negCls

In [ ]:
def save_neg_fig(pth_list):
  fig,ax = plt.subplots(1)
  fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
  ax.axis('off')
  for i in pth_list.index:
    ax.specgram(df.iloc[i, :].mfccs, Fs = rate, cmap = plt.cm.bone)
    fig.savefig('./negClss/'+df.iloc[i, :].id, bbox_inches='tight', pad_inches = 0)
    
def save_pos_fig(pth_list):
  fig,ax = plt.subplots(1)
  fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
  ax.axis('off')
  for i in pth_list.index:
    ax.specgram(df.iloc[i, :].mfccs, Fs = rate, cmap = plt.cm.bone)
    fig.savefig('./posClss/'+df.iloc[i, :].id, bbox_inches='tight', pad_inches = 0)

In [ ]:
mfcc_pos_lis = df[df.test_status=='p']
mfcc_neg_lis = df[df.test_status=='n']

In [ ]:
for i in df[df.test_status=='p'].index:
 pass 
print(df.iloc[i, :].id)

### Positive Figures

In [ ]:
!mkdir negClss
!mkdir posClss

In [ ]:
save_neg_fig(mfcc_neg_lis)

In [ ]:
!mv ./*.png ./negClss

In [ ]:
save_pos_fig(mfcc_pos_lis)

In [ ]:
!mv ./*.png ./posClss

In [ ]:
!zip -r ./posClss.zip ./posClss

In [ ]:
!zip -r ./negClss.zip ./negClss

In [ ]:
!cp -r /content/positive_mel2 '/content/drive/MyDrive/Colab Notebooks/Coswara'

In [ ]:
!rm -r /content/positive_mel2

In [ ]:
!zip -r ./posCls.zip ./posCls

### Negative Figures

In [ ]:
save_fig(mfcc_neg_lis)

In [ ]:
!mkdir negative_mel2
!mv /content/*.png /content/negative_mel2

In [ ]:
!cp -r /content/negative_mel2 '/content/drive/MyDrive/Colab Notebooks/Coswara'

In [ ]:
!rm -r /content/negative_mel2

# Model Training

In [ ]:
!cp -r '/content/drive/MyDrive/Colab Notebooks/Coswara' /content/

In [ ]:
import cv2
import glob

pos_images=[]
neg_images=[]

for imagepath in glob.glob("/content/Coswara/Positive/positive_mfcc1/*.png"):
        img=cv2.imread(imagepath,0)
        # img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                         
        img=cv2.resize(img,(224,224))
        pos_images.append(img)
print(len(pos_images))

for imagepath in glob.glob("/content/Coswara/Negative/negative_mfcc1/*.png"):
        img=cv2.imread(imagepath,0)
        # img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                         
        img=cv2.resize(img,(224,224))
        neg_images.append(img)
print(len(neg_images))

In [ ]:
!rm -r /content/Coswara/test
!rm -r /content/Coswara/train
!rm -r /content/Coswara/val

In [ ]:
# os.listdir('/content/Coswara/Positive/positive_mfcc1/')
for i in [1,2,3]:
  print(i)

In [ ]:
import os
import numpy as np
import shutil

# # Creating Train / Val / Test folders (One time use)
root_dir = '.'
posCls = '/posCls'
negCls = '/negCls'

if 'train' not in os.listdir('./'):
  os.makedirs(root_dir +'/train' + posCls)
  os.makedirs(root_dir +'/train' + negCls)
  os.makedirs(root_dir +'/val' + posCls)
  os.makedirs(root_dir +'/val' + negCls)
  os.makedirs(root_dir +'/test' + posCls)
  os.makedirs(root_dir +'/test' + negCls)

# Creating partitions of the data after shuffeling
currentCls = posCls
src1 = "."+posCls # Folder to copy images from
src2 = "."+negCls

for src in [src1, src2]:
  allFileNames = os.listdir(src)
  np.random.shuffle(allFileNames)
  train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)*0.7), int(len(allFileNames)*0.85)])

  train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
  val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
  test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
print('Validation: ', len(val_FileNames))
print('Testing: ', len(test_FileNames))

# Copy-pasting images
for currentCls in ['/posCls', '/negCls']:
    for name in train_FileNames:
        shutil.copy(name, "./train"+currentCls)

    for name in val_FileNames:
        shutil.copy(name, "./val"+currentCls)

    for name in test_FileNames:
        shutil.copy(name, "./test"+currentCls)

In [ ]:
import PIL
import PIL.Image
import tensorflow as tf

In [ ]:
batch_size = 16
img_height = 224
img_width = 224
traindata_dir = './train'
testdata_dir = './test'
valdata_dir = './val'

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  traindata_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  testdata_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  testdata_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
normalized_ds

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  # tf.keras.Input(shape=(224, 224, 3), batch_size=16),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=10
)

In [ ]:
model.predict(val_ds)

In [ ]:
model.evaluate(val_ds)

In [ ]:
val_ds.class_names

### Pre-trained


https://www.kaggle.com/himanshu007121/coughclassifier-trial

In [ ]:
from keras.models import *
from keras.layers import *
import tensorflow as tf

# Load the pretained model
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='softmax')(x)

model1 = tf.keras.Model(inputs=inputs, outputs=outputs)

model1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model1.summary()

In [ ]:
history = model1.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

### 3D
https://www.machinecurve.com/index.php/2019/10/18/a-simple-conv3d-example-with-keras/


In [ ]:
sample_shape = (1, 224, 224, 3, 1)
# Create the model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(no_classes, activation='softmax'))